# **Import Modules**

In [3]:
# TF --> TPU environment version
!pip install -U tensorflow==2.4.1

     |████████████████████████████████| 394.3 MB 7.2 kB/s               |██████████████▏                 | 174.0 MB 18.1 MB/s eta 0:00:13
     |████████████████████████████████| 2.9 MB 40.4 MB/s            
     |████████████████████████████████| 3.8 MB 52.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 462 kB 35.3 MB/s            
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77049 sha256=6e0e5170eadfe435140aacd208cdf42a449c7b3985314f8c5aeaa9cf5e770944
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successful

In [4]:
import sys

import os

import copy

import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.insert(0, '../input/deeplearning-utils')

In [5]:
# from utils.external.common import Reader

from utils.tf.ops import io

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import tensorflow as tf

from tensorflow.keras import backend, optimizers, regularizers

from tensorflow.keras.layers import *

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.4.1


# **Config**

In [8]:
class Config:
    
    name = 'vgg16'
    
    IMAGE_SIZE = (224, 224, 3)

# **Build Model**

In [9]:
def adjust_name(name):
    
    if name is None:
    
        name = ''
    
    else:
        
        name = name + '_'
        
    return name

In [10]:
def conv2d_block(inputs, filters, use_conv3=False, name=None):
    
    name = adjust_name(name)
     
    kernel_size = (3, 3)
    
    z = Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', name=f'{name}conv1')(inputs)
    z = Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', name=f'{name}conv2')(z)
    
    if use_conv3:
        
        z = Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', name=f'{name}conv3')(z)
    
    z = MaxPooling2D((2, 2), strides=(2, 2), name=f'{name}pool')(z)
    
    return z

In [11]:
def build_blocks(inputs):
    
    # 1
    z = conv2d_block(inputs, 64, use_conv3=False, name='block1')

    # 2
    z = conv2d_block(z, 128, use_conv3=False, name='block2')

    # 3
    z = conv2d_block(z, 256, use_conv3=True, name='block3')

    # 4
    z = conv2d_block(z, 512, use_conv3=True, name='block4')

    # 5
    z = conv2d_block(z, 512, use_conv3=True, name='block5')
    
    return z

In [12]:
def top_block(inputs, num_classes, dropout_rate=None):
    
    z = Flatten(name='flatten')(inputs)
    
    z = Dense(4096, activation='relu', name='fc1')(z)
    
    if dropout_rate is not None:
        
        z = Dropout(dropout_rate)(z)
    
    z = Dense(4096, activation='relu', name='fc2')(z)
    
    if dropout_rate is not None:
        
        z = Dropout(dropout_rate)(z)
        
    y = Dense(num_classes, activation='softmax', name='predictions')(z)
    
    return y

In [19]:
def build_model(include_top, num_classes, dropout_rate, name=None):
    
    inputs = Input(shape=Config.IMAGE_SIZE)

    outputs = build_blocks(inputs)
    
    if include_top:
        
        outputs = top_block(outputs, num_classes, dropout_rate)

    vgg16 = Model(inputs, outputs, name=name)
    
    return vgg16

# **Load Pretrained Weight**

In [20]:
CHECKPOINTS = {
    'imagenet_no_top': ('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/'),
    'imagenet_top': ('vgg16_weights_tf_dim_ordering_tf_kernels.h5', 'https://storage.googleapis.com/tensorflow/keras-applications/vgg16'),
    'places365_no_top': ('vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5', 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/'),
    'places365_top': ('vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5', 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/')
}

In [21]:
def download_checkpoints(key):
    
    fname = CHECKPOINTS[key][0]
    
    url = f'{CHECKPOINTS[key][1]}/{fname}'
    
    download_path = io.download(url=url, fname=fname)
    
    return download_path

# **Export Model**

In [29]:
key = 'imagenet_no_top'

vgg16 = build_model(include_top=False, num_classes=1000, dropout_rate=None)

download_path = download_checkpoints(key)

vgg16.load_weights(download_path)

vgg16.save(f'vgg16_{key}.h5')

In [30]:
key = 'imagenet_top'

vgg16 = build_model(include_top=True, num_classes=1000, dropout_rate=None)

download_path = download_checkpoints(key)

vgg16.load_weights(download_path)

vgg16.save(f'vgg16_{key}.h5')

In [31]:
key = 'places365_no_top'

vgg16 = build_model(include_top=False, num_classes=365, dropout_rate=0.5)

download_path = download_checkpoints(key)

vgg16.load_weights(download_path)

vgg16.save(f'vgg16_{key}.h5')

In [32]:
key = 'places365_top'

vgg16 = build_model(include_top=True, num_classes=365, dropout_rate=0.5)

download_path = download_checkpoints(key)

vgg16.load_weights(download_path)

vgg16.save(f'vgg16_{key}.h5')